In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import seaborn as sns

from scipy.stats import pearsonr

sns.set_style("darkgrid")
np.random.seed(930525)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)

warnings.simplefilter('once')

%matplotlib inline
%load_ext watermark
%watermark --iversions

numpy   1.19.5
seaborn 0.11.1
pandas  1.1.4



In [2]:
import networkx as nx
import statsmodels.formula.api as smf

/home/bhillmann/.conda/envs/type_1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/bhillmann/.conda/envs/type_1/lib/python3.8/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping


In [3]:
df_dag_train = pd.read_csv("../results/dag.prediction.train.response.updated.csv", sep="\t")
df_dag_train_warp = pd.read_csv("../results/dag.prediction.train.response.warp.updated.csv", sep="\t")
df_train = pd.read_csv("../data/train.txt", index_col=0, header = 0, sep="\t")
df_train_warp = pd.read_csv("../data/train.warp.txt", index_col=0, header = 0, sep="\t")
df_test = pd.read_csv("../data/test.txt", index_col=0, header = 0, sep="\t")
df_test_warp = pd.read_csv("../data/test.warp.txt", index_col=0, header = 0, sep="\t")

# df_train.columns = [col.replace(" ", "_").replace(";", ".") for col in df_train.columns]

/home/bhillmann/.conda/envs/type_1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df_results_warp = pd.read_csv("../data/results.dbn.warp.txt", sep="\t")
df_results_model = pd.read_csv("../data/results.dbn.txt", sep="\t")

In [5]:
day_plus_one = [col for col in df_dag_train.columns if "day_plus_one" in col]

In [6]:
s_binary = {"Gender", "Supplement"}

In [7]:
G = nx.DiGraph()

# linear models warp
d_response_2_model_warp = {}
for response in day_plus_one:
    features = df_dag_train_warp[response]
    significant_features = features[features > 0]
    fn_count = np.sum([(("nutrient" in _) or ("food" in _)) for _ in significant_features.index])
    if significant_features.sum() > 1 and fn_count > 0:
        model_str = "Q('" + response + "')" + " ~ " + " + ".join(["Q('" + _ + "')"  if _ not in s_binary else "C(Q('" + _ + "'))" for _ in significant_features.index]) + " + 1"
        model = smf.ols(formula=model_str, data=df_train_warp)
        res = model.fit_regularized(method="elastic_net", L1_wt=0)
        d_response_2_model_warp[response] = res
        response_clean = "t+1." + response.split(".")[-1]
        G.add_nodes_from([
            (response_clean, {"color": "orange", "shape": response.split(".")[1]})
        ])
        for ix, significant_feature in enumerate(significant_features.index):
            significant_feature_clean = significant_feature.split(".")[-1]
            if significant_feature_clean not in G:
                G.add_nodes_from([
                    (significant_feature_clean, {"color": "blue", "shape": significant_feature.split(".")[0]})
                ])
            # Get coefficient from model
            G.add_edge(significant_feature_clean, response_clean)
            G[significant_feature_clean][response_clean]["color"] = "green" if res.params[ix + 1] > 0 else "red"

In [8]:
nx.write_gml(G, "network.gml")

/home/bhillmann/.conda/envs/type_1/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
res.params

array([-0.6758818 ,  0.10773751,  0.70284322, -0.33273872, -0.22729032])

In [10]:
res.model.exog_names

['Intercept',
 "Q('tax.k__Bacteria.p__Verrucomicrobiota.c__Verrucomicrobiae.o__Verrucomicrobiales.f__Akkermansiaceae.g__Akkermansia.s__Akkermansia_muciniphila')",
 "Q('tax.k__Bacteria.p__Verrucomicrobiota.c__Verrucomicrobiae.o__Verrucomicrobiales.f__Akkermansiaceae.g__Akkermansia.s__Akkermansia_muciniphila_B')",
 "Q('food.L1_Meat_Poultry_Fish_and_Mixtures.L2_Frozen_and_shelfstable_plate_meals_soups_and_gravies.L3_Soups_broths_extracts_from_meat_poultry_fish_base')",
 "Q('food.L1_Meat_Poultry_Fish_and_Mixtures.L2_Meatpoultry_fish_with_nonmeat.L3_Sandwiches_with_meat_poultry_fish')"]

In [11]:
res.model.exog_names

['Intercept',
 "Q('tax.k__Bacteria.p__Verrucomicrobiota.c__Verrucomicrobiae.o__Verrucomicrobiales.f__Akkermansiaceae.g__Akkermansia.s__Akkermansia_muciniphila')",
 "Q('tax.k__Bacteria.p__Verrucomicrobiota.c__Verrucomicrobiae.o__Verrucomicrobiales.f__Akkermansiaceae.g__Akkermansia.s__Akkermansia_muciniphila_B')",
 "Q('food.L1_Meat_Poultry_Fish_and_Mixtures.L2_Frozen_and_shelfstable_plate_meals_soups_and_gravies.L3_Soups_broths_extracts_from_meat_poultry_fish_base')",
 "Q('food.L1_Meat_Poultry_Fish_and_Mixtures.L2_Meatpoultry_fish_with_nonmeat.L3_Sandwiches_with_meat_poultry_fish')"]

In [12]:
G.add_nodes_from([
    ("test", {"color": "orange"})
    ("test2", {"color": "blue"}),
])

<>:2: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?


TypeError: 'tuple' object is not callable

In [ ]:
G.add_edge("test", "test2")
G["test"]["test2"]["color"] = "green"


In [ ]:
nx.write_gml(G, "network.gml")

In [ ]:
d_response_2_model_warp